# Retail Analytics Case — Segmentation Magasins & Dimensionnement d’Opportunité

Objectif : analyser la performance des magasins de manière **descriptive** (sur les données disponibles), segmenter le parc, estimer un potentiel « si un magasin atteignait le benchmark », puis prioriser les actions.

---

## Sources de données
- `store_kpis.csv` : KPI pré-agrégés au niveau magasin
- `train.csv` + `transactions.csv` : ventes et trafic journaliers (utilisés pour l’UPT période et certaines mesures de potentiel)
- `stores.csv` : optionnel (libellés ville)

## Définitions clés
- **UPT** (*Unités par Transaction*) : `units / transactions`
- **Segmentation** (au sein de chaque *type* de magasin) :  
  `basket_index = basket_units / median(basket_units | type)`  
  - ≤ P20 → **Alerte (Sous-performance)**
  - ≥ P80 → **Leaders (Sur-performance)**
  - sinon → **Standard**

> Note : le « potentiel » est un **scénario** (what-if) basé sur un benchmark descriptif ; il ne démontre pas une relation causale.


In [1]:
# Core libraries
from __future__ import annotations

from pathlib import Path

import numpy as np
import pandas as pd

import duckdb
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.colors import sample_colorscale

# Pandas display (optional)
pd.set_option("display.max_columns", 60)
pd.set_option("display.width", 140)

# -------------------------
# Paths (relative to this notebook)
# -------------------------
DATA_DIR = Path("..") / "data"
KPI_PATH = DATA_DIR / "processed" / "store_kpis.csv"
TRAIN_PATH = DATA_DIR / "raw" / "train.csv"
TX_PATH = DATA_DIR / "raw" / "transactions.csv"
STORES_PATH = DATA_DIR / "raw" / "stores.csv"

REPORTS_DIR = Path("..") / "reports"
OUTPUT_DASHBOARD = REPORTS_DIR / "dashboard_retail.html"

# -------------------------
# Business constants
# -------------------------
SEGMENT_ORDER = [
    "Alerte (Sous-performance)",
    "Standard",
    "Leaders (Sur-performance)",
]

# -------------------------
# Helpers
# -------------------------
def compute_segments(df_kpi: pd.DataFrame) -> pd.DataFrame:
    """Compute store segmentation based on basket_index quantiles (P20/P80), within each store type."""
    out = df_kpi.copy()

    type_med = out.groupby("type")["basket_units"].median()
    out["type_median_basket"] = out["type"].map(type_med)

    out["basket_index"] = out["basket_units"] / out["type_median_basket"]

    p20 = out["basket_index"].quantile(0.20)
    p80 = out["basket_index"].quantile(0.80)

    out["segment"] = np.where(
        out["basket_index"] <= p20, SEGMENT_ORDER[0],
        np.where(out["basket_index"] >= p80, SEGMENT_ORDER[2], SEGMENT_ORDER[1])
    )

    return out


def read_csv_checked(path: Path, *, usecols=None, dtype=None) -> pd.DataFrame:
    """Read CSV with lightweight safeguards and clearer errors."""
    if not path.exists():
        raise FileNotFoundError(f"File not found: {path.resolve()}")
    return pd.read_csv(path, usecols=usecols, dtype=dtype)


print("Environment ready.")


Environment ready.


## 1) Chargement des KPI magasins et segmentation

On charge `store_kpis.csv`, on calcule le `basket_index` (vs médiane du format) et on attribue un segment par magasin.


In [2]:
# Load KPIs (store-level)
df_kpi = read_csv_checked(KPI_PATH)

# Ensure segmentation is present and consistent
if "segment" not in df_kpi.columns:
    df_kpi = compute_segments(df_kpi)
else:
    # If segment already exists, we still ensure required derived fields exist
    required_cols = {"type_median_basket", "basket_index"}
    if not required_cols.issubset(df_kpi.columns):
        df_kpi = compute_segments(df_kpi)

# Optional enrichment: city label (used in hover/labels)
stores_df = None
if STORES_PATH.exists():
    stores_df = read_csv_checked(
        STORES_PATH,
        usecols=["store_nbr", "city"],
        dtype={"store_nbr": "int32", "city": "string"},
    )
    if "city" not in df_kpi.columns:
        df_kpi = df_kpi.merge(stores_df, on="store_nbr", how="left")
elif "city" not in df_kpi.columns:
    df_kpi["city"] = "Magasin"

# Quick sanity check
df_kpi["segment"].value_counts()


segment
Standard                     32
Alerte (Sous-performance)    11
Leaders (Sur-performance)    11
Name: count, dtype: int64

## 2) Estimation rapide du potentiel par magasin (proxy d’écart panier)

Estimation descriptive d’un écart de panier vs la médiane du format (clippé à 0), puis conversion en unités potentielles via le trafic moyen et le nombre de jours actifs.


In [3]:
# Basket gap vs type median (clipped at 0 => no "negative opportunity")
df_kpi["gap_basket"] = (df_kpi["type_median_basket"] - df_kpi["basket_units"]).clip(lower=0)

# Convert gap into potential units (descriptive proxy)
df_kpi["opp_units_per_day"] = df_kpi["gap_basket"] * df_kpi["avg_transactions"]
df_kpi["opp_units_total"] = df_kpi["opp_units_per_day"] * df_kpi["active_days"]

# Top stores by daily opportunity (for review)
(
    df_kpi.sort_values("opp_units_per_day", ascending=False)
    .head(10)[
        [
            "store_nbr",
            "city",
            "type",
            "segment",
            "avg_transactions",
            "basket_units",
            "type_median_basket",
            "opp_units_per_day",
        ]
    ]
)


,store_nbr,city,type,segment,avg_transactions,basket_units,type_median_basket,opp_units_per_day
33,34,Guayaquil,B,Alerte (Sous-performance),2302.752086,4.418764,7.587222,7296.172466
45,46,Quito,A,Alerte (Sous-performance),3571.921884,6.979294,8.519189,5500.382809
49,50,Ambato,A,Alerte (Sous-performance),2614.456768,6.522452,8.519189,5220.381076
47,48,Quito,A,Alerte (Sous-performance),3045.787120,7.018842,8.519189,4569.736892
0,1,Quito,D,Alerte (Sous-performance),1523.844272,5.532976,7.487447,2978.308702
46,47,Quito,A,Standard,3897.322600,7.778360,8.519189,2887.249334
7,8,Quito,D,Standard,2767.285800,6.556312,7.487447,2576.716855
13,14,Riobamba,C,Alerte (Sous-performance),1396.733822,4.964678,6.671998,2384.670815
42,43,Esmeraldas,E,Alerte (Sous-performance),1305.236842,7.493645,9.085206,2077.363929
14,15,Ibarra,C,Alerte (Sous-performance),1318.554893,5.167658,6.671998,1983.553648


## 3) Graphique 1 — Paysage de segmentation (Trafic vs Ventes)

Cartographie du parc : trafic moyen vs ventes moyennes, couleur = segment.


In [5]:
fig1 = px.scatter(
    df_kpi,
    x="avg_transactions",
    y="avg_daily_sales",
    color="segment",
    size="avg_transactions",
    hover_name="city",
    color_discrete_map={
        "Leaders (Sur-performance)": "#0F9D58",
        "Standard": "#F4B400",
        "Alerte (Sous-performance)": "#DB4437",
    },
    title="<b> Performance vs Trafic : Identification des Magasins Critiques</b>",
    labels={
        "avg_transactions": "Transactions Moyennes / Jour",
        "avg_daily_sales": "Ventes Moyennes (Unités) / Jour",
    },
)
fig1.update_layout(template="plotly_white", height=500)
fig1.show()


## 4) Agrégation période (train + transactions) et benchmark par type

Reconstruction des agrégats période par magasin (unités, transactions) à partir des données journalières, puis calcul d’un benchmark UPT par **type** basé sur les **Leaders**.


In [6]:
# Store lookup: segment + type (+ city if available)
seg_lookup = df_kpi[["store_nbr", "type", "segment"]].copy()

if stores_df is None and STORES_PATH.exists():
    stores_df = read_csv_checked(
        STORES_PATH,
        usecols=["store_nbr", "city"],
        dtype={"store_nbr": "int32", "city": "string"},
    )

if stores_df is not None:
    seg_lookup = seg_lookup.merge(stores_df, on="store_nbr", how="left")
else:
    seg_lookup["city"] = "Magasin"

# Load daily sales + daily traffic with minimal columns (memory-friendly dtypes)
train = read_csv_checked(
    TRAIN_PATH,
    usecols=["date", "store_nbr", "sales"],
    dtype={"date": "string", "store_nbr": "int32", "sales": "float64"},
)
tx = read_csv_checked(
    TX_PATH,
    usecols=["date", "store_nbr", "transactions"],
    dtype={"date": "string", "store_nbr": "int32", "transactions": "int32"},
)

# Daily units per store
daily_units = (
    train.groupby(["date", "store_nbr"], as_index=False)["sales"]
    .sum()
    .rename(columns={"sales": "units"})
)

# Join units + transactions (inner join ensures aligned days)
daily = daily_units.merge(tx, on=["date", "store_nbr"], how="inner")
daily = daily[daily["transactions"] > 0].copy()

# Period aggregates per store
store_period = daily.groupby("store_nbr", as_index=False).agg(
    units=("units", "sum"),
    transactions=("transactions", "sum"),
)

store_period = (
    store_period.merge(seg_lookup, on="store_nbr", how="left")
    .dropna(subset=["segment", "type"])
)

# Type benchmark UPT from Leaders
leaders = store_period[store_period["segment"] == SEGMENT_ORDER[2]].copy()

bench_type = (
    leaders.groupby("type", as_index=False)
    .apply(lambda g: pd.Series({"bench_upt": g["units"].sum() / g["transactions"].sum()}))
    .reset_index(drop=True)
)

store_period = store_period.merge(bench_type, on="type", how="left").dropna(subset=["bench_upt"])
store_period["expected_units"] = store_period["transactions"] * store_period["bench_upt"]
store_period["opportunity_units"] = (store_period["expected_units"] - store_period["units"]).clip(lower=0)

store_period.head()


,store_nbr,units,transactions,type,segment,city,bench_upt,expected_units,opportunity_units
0,1,1.413102e+07,2553963,D,Alerte (Sous-performance),Quito,9.404979,2.401997e+07,9.888952e+06
1,2,2.151546e+07,3219901,D,Standard,Quito,9.404979,3.028310e+07,8.767642e+06
2,3,5.033053e+07,5366350,D,Leaders (Sur-performance),Quito,9.404979,5.047041e+07,1.398834e+05
3,4,1.885445e+07,2519007,D,Standard,Quito,9.404979,2.369121e+07,4.836754e+06
4,5,1.556692e+07,2347877,D,Standard,Santo Domingo,9.404979,2.208173e+07,6.514817e+06


## 5) Graphique 2 — Performance vs potentiel (UPT) par segment

Comparaison UPT réel vs UPT benchmark (Leaders) par segment, avec mise en évidence du gap sur **Alerte**.


In [8]:
# Aggregate by segment (period totals)
seg_sum = (
    store_period.groupby("segment", as_index=False)
    .agg(
        units=("units", "sum"),
        expected_units=("expected_units", "sum"),
        transactions=("transactions", "sum"),
        opportunity=("opportunity_units", "sum"),
    )
)

# Recompute UPT KPIs
seg_sum["upt_actual"] = seg_sum["units"] / seg_sum["transactions"]
seg_sum["upt_bench"] = seg_sum["expected_units"] / seg_sum["transactions"]

# Ensure segment order
seg_sum["segment"] = pd.Categorical(seg_sum["segment"], categories=SEGMENT_ORDER, ordered=True)
seg_sum = seg_sum.sort_values("segment")

# Long format for Plotly overlay bars
seg_long = seg_sum.melt(
    id_vars=["segment", "opportunity"],
    value_vars=["upt_actual", "upt_bench"],
    var_name="metric",
    value_name="upt",
)

seg_long["metric"] = seg_long["metric"].map(
    {"upt_actual": "UPT réel", "upt_bench": "UPT benchmark (Leaders)"}
)

# Total opportunity for Alerte (used in annotation)
alerte_row = seg_sum[seg_sum["segment"] == SEGMENT_ORDER[0]]
alerte_opp = float(alerte_row["opportunity"].iloc[0]) if not alerte_row.empty else 0.0

# Colors
color_real = "#2E86C1"
color_bench = "#D6DBDF"

fig2 = go.Figure()

# Benchmark bar (background)
fig2.add_trace(
    go.Bar(
        x=seg_long[seg_long["metric"] == "UPT benchmark (Leaders)"]["segment"],
        y=seg_long[seg_long["metric"] == "UPT benchmark (Leaders)"]["upt"],
        name="Potentiel (Benchmark)",
        marker_color=color_bench,
        width=0.6,
        text=seg_long[seg_long["metric"] == "UPT benchmark (Leaders)"]["upt"],
        texttemplate="%{text:.2f}",
        textposition="outside",
        textfont=dict(color="#7F8C8D"),
    )
)

# Actual bar (foreground)
fig2.add_trace(
    go.Bar(
        x=seg_long[seg_long["metric"] == "UPT réel"]["segment"],
        y=seg_long[seg_long["metric"] == "UPT réel"]["upt"],
        name="Performance Actuelle",
        marker_color=color_real,
        width=0.35,
        text=seg_long[seg_long["metric"] == "UPT réel"]["upt"],
        texttemplate="%{text:.2f}",
        textposition="auto",
        textfont=dict(color="white"),
    )
)

fig2.update_layout(
    title={
        "text": "<b>PERFORMANCE VS POTENTIEL:</b> Analyse du Panier Moyen (UPT)",
        "y": 0.95,
        "x": 0.5,
        "xanchor": "center",
        "yanchor": "top",
    },
    template="plotly_white",
    barmode="overlay",
    height=550,
    width=1000,
    legend=dict(orientation="h", y=1.02, x=1, xanchor="right"),
    yaxis=dict(
        title="Unités par Transaction",
        showgrid=True,
        gridcolor="#F4F6F6",
        zeroline=False,
        showticklabels=False,
    ),
    xaxis=dict(showgrid=False),
    margin=dict(t=100, l=50, r=50, b=50),
)

# Gap annotation (Alerte)
try:
    alerte_bench = seg_long[
        (seg_long["segment"] == SEGMENT_ORDER[0]) & (seg_long["metric"] == "UPT benchmark (Leaders)")
    ]["upt"].values[0]
    alerte_real = seg_long[
        (seg_long["segment"] == SEGMENT_ORDER[0]) & (seg_long["metric"] == "UPT réel")
    ]["upt"].values[0]
    gap_val = alerte_bench - alerte_real

    fig2.add_annotation(
        x=SEGMENT_ORDER[0],
        y=alerte_bench,
        text=(
            f"<b>GAP CRITIQUE: -{gap_val:.1f}</b><br>"
            f"<span style='font-size:11px; color:#555'>Ce gap représente<br><b>{alerte_opp:,.0f} unités</b> (scénario)</span>"
        ),
        showarrow=True,
        arrowhead=2,
        ax=70,
        ay=0,
        bgcolor="#FFF",
        bordercolor="#E74C3C",
        borderwidth=1,
        font=dict(color="#C0392B"),
    )
except IndexError:
    print("Warning: not enough data to annotate the Alerte gap.")

fig2.show()


## 6) Graphique 3 — Diagnostic mix produit (écart pondéré vs Leaders)

Comparaison des parts de ventes par famille entre **Leaders** et **Alerte**, pondérée par le poids des Leaders (par type).


In [10]:
# Reuse a fresh in-memory connection for clarity (fast and isolated)
con = duckdb.connect(database=":memory:")
con.register("seg_lookup", df_kpi[["store_nbr", "type", "segment"]])

mix_sql = f'''
WITH base AS (
    SELECT s.type, s.segment, t.family, SUM(t.sales) AS units
    FROM read_csv_auto('{TRAIN_PATH.as_posix()}') t
    JOIN seg_lookup s ON t.store_nbr = s.store_nbr
    GROUP BY 1,2,3
),
shares AS (
    SELECT type, segment, family, units,
           units / NULLIF(SUM(units) OVER (PARTITION BY type, segment), 0) AS share
    FROM base
),
type_weights AS (
    SELECT type, SUM(units) AS leaders_type_units
    FROM base
    WHERE segment = '{SEGMENT_ORDER[2]}' -- Leaders
    GROUP BY 1
),
pivoted AS (
    SELECT type, family,
        MAX(CASE WHEN segment = '{SEGMENT_ORDER[2]}' THEN share END) AS share_leaders,
        MAX(CASE WHEN segment = '{SEGMENT_ORDER[0]}' THEN share END) AS share_alerte
    FROM shares GROUP BY 1,2
),
typed_delta AS (
    SELECT p.type, p.family,
        (p.share_leaders - p.share_alerte) * 100.0 AS delta_pp,
        w.leaders_type_units
    FROM pivoted p
    JOIN type_weights w USING(type)
    WHERE p.share_leaders IS NOT NULL
      AND p.share_alerte  IS NOT NULL
      AND w.leaders_type_units > 0
),
weighted AS (
    SELECT family,
        SUM(delta_pp * leaders_type_units) / NULLIF(SUM(leaders_type_units), 0) AS weighted_gap_pp
    FROM typed_delta
    GROUP BY 1
)
SELECT family, weighted_gap_pp
FROM weighted
WHERE weighted_gap_pp >= 0.1
ORDER BY weighted_gap_pp DESC
LIMIT 10;
'''

mix_data = con.execute(mix_sql).df()

# French retail category names for portfolio readability (display only)
FAMILY_FR_MAP = {
    "PRODUCE": "Fruits & légumes",
    "POULTRY": "Volaille",
    "MEATS": "Boucherie",
    "SEAFOOD": "Poissonnerie",
    "DAIRY": "Produits laitiers",
    "EGGS": "Œufs",
    "BREAD/BAKERY": "Boulangerie & pâtisserie",
    "DELI": "Traiteur",
    "PREPARED FOODS": "Plats préparés",
    "BEVERAGES": "Boissons",
    "LIQUOR,WINE,BEER": "Alcools, vin & bière",
    "FROZEN FOODS": "Surgelés",
    "GROCERY I": "Épicerie I",
    "GROCERY II": "Épicerie II",
    "CLEANING": "Entretien ménager",
    "HOME CARE": "Entretien maison",
    "HOME AND KITCHEN": "Maison & cuisine",
    "PERSONAL CARE": "Hygiène & beauté",
    "BABY CARE": "Bébé",
    "PET SUPPLIES": "Animalerie",
    "LADIESWEAR": "Mode femme",
    "LAWN AND GARDEN": "Jardin",
    "SCHOOL AND OFFICE SUPPLIES": "Papeterie",
    "AUTOMOTIVE": "Automobile",
    "MAGAZINES": "Magazines",
    "CELEBRATION": "Fêtes",
    "BEAUTY": "Beauté",
}

if mix_data.empty:
    print("No mix gap >= 0.1 pp found.")
else:
    mix_data = mix_data.sort_values("weighted_gap_pp", ascending=True)
    mix_data["family_fr"] = mix_data["family"].map(FAMILY_FR_MAP).fillna(mix_data["family"])

    fig4 = px.bar(
        mix_data,
        x="weighted_gap_pp",
        y="family_fr",
        orientation="h",
        template="plotly_white",
        color="weighted_gap_pp",
        color_continuous_scale="Reds",
        text=mix_data["weighted_gap_pp"].map(lambda v: f"+{v:.2f} pp"),
        title=(
            "<b> Diagnostic Mix Produit (Top Opportunités)</b>"
            "<br><sup>Catégories avec un déficit significatif par rapport aux Leaders (Pondéré)</sup>"
        ),
    )

    fig4.update_layout(
        showlegend=False,
        coloraxis_showscale=False,
        height=550,
        width=1050,
        margin=dict(l=160, r=40, t=100, b=110),
        xaxis_title="Gap de Mix (points de pourcentage)",
        yaxis_title="",
        yaxis=dict(categoryorder="array", categoryarray=mix_data["family_fr"].tolist()),
    )

    fig4.update_traces(
        textposition="outside",
        cliponaxis=False,
        marker_line_width=0,
        width=0.7,
    )

    fig4.add_vline(x=0, line_width=1, line_color="#D5D8DC")

    fig4.add_annotation(
        x=0,
        y=-0.25,
        xref="paper",
        yref="paper",
        showarrow=False,
        align="left",
        xanchor="left",
        text="<i>*Note: écart descriptif pondéré (indique un potentiel, pas une preuve causale).</i>",
        font=dict(size=11, color="#7F8C8D"),
    )

    fig4.show()


## 7) Graphique 4 — Plan d’action (Pareto) sur les magasins Alerte

Priorisation des magasins du segment **Alerte** en fonction de l’opportunité (unités) — vue Pareto.


In [11]:
# Focus on Alerte stores
focus = store_period[store_period["segment"] == SEGMENT_ORDER[0]].copy()
focus = focus.sort_values("opportunity_units", ascending=False)

total_opp = float(focus["opportunity_units"].sum())

if total_opp == 0:
    print("No non-zero opportunity detected (based on the available data).")
else:
    TOP_N = 6
    top = focus.head(TOP_N).copy()
    others_opp = float(focus.iloc[TOP_N:]["opportunity_units"].sum())

    def make_label(row: pd.Series) -> str:
        city_str = str(row["city"]) if pd.notna(row["city"]) else "Magasin"
        return f"{city_str}<br>M{int(row['store_nbr'])} ({row['type']})"

    top["label"] = top.apply(make_label, axis=1)

    if others_opp > 0:
        others_row = pd.DataFrame(
            [
                {
                    "label": "Autres<br>Magasins",
                    "opportunity_units": others_opp,
                    "is_other": True,
                }
            ]
        )
        top = pd.concat([top, others_row], ignore_index=True)
    else:
        top["is_other"] = False

    top["cum_opp_share"] = top["opportunity_units"].cumsum() / total_opp

    last_top_idx = TOP_N - 1
    anno_anchor_x = top.loc[last_top_idx, "label"]
    anno_anchor_y = float(top.loc[last_top_idx, "cum_opp_share"])

    # Colors
    BAR_GRADIENT_START = 0.5
    BAR_GRADIENT_END = 0.9
    COLOR_OTHERS = "#CFD8DC"
    COLOR_LINE = "#37474F"

    fig5 = go.Figure()

    n_core = len(top) - (1 if others_opp > 0 else 0)
    reds_scale = sample_colorscale("Reds", np.linspace(BAR_GRADIENT_END, BAR_GRADIENT_START, n_core))
    colors = reds_scale + ([COLOR_OTHERS] if others_opp > 0 else [])

    fig5.add_trace(
        go.Bar(
            x=top["label"],
            y=top["opportunity_units"],
            name="Opportunité",
            marker=dict(color=colors, line=dict(width=0)),
            text=top["opportunity_units"],
            texttemplate="%{text:,.0f}",
            textposition="outside",
            cliponaxis=False,
            hovertemplate="Opportunité: %{y:,.0f}<extra></extra>",
        )
    )

    fig5.add_trace(
        go.Scatter(
            x=top["label"],
            y=top["cum_opp_share"],
            name="Cumul %",
            yaxis="y2",
            mode="lines+markers",
            line=dict(color=COLOR_LINE, width=2, dash="dot"),
            marker=dict(color=COLOR_LINE, size=8),
            hovertemplate="Cumul: %{y:.0%}<extra></extra>",
        )
    )

    fig5.add_annotation(
        x=anno_anchor_x,
        y=anno_anchor_y,
        xref="x",
        yref="y2",
        text=f"Ce Top {TOP_N} concentre <b>{anno_anchor_y:.0%}</b><br>de l'opportunité totale.",
        showarrow=True,
        arrowhead=1,
        arrowsize=1,
        arrowwidth=1,
        arrowcolor=COLOR_LINE,
        ax=-60,
        ay=-50,
        bgcolor="rgba(255,255,255,0.9)",
        borderpad=4,
        font=dict(size=11, color=COLOR_LINE),
    )

    fig5.update_layout(
        template="plotly_white",
        height=580,
        width=1000,
        margin=dict(l=60, r=60, t=100, b=100),
        title=(
            "<b> Plan d'Action : Priorisation (Pareto)</b>"
            "<br><sup>Focalisation sur le Top 6 magasins. Benchmark ajusté par format.</sup>"
        ),
        xaxis=dict(title="", tickangle=0, automargin=True, type="category"),
        yaxis=dict(showgrid=False, showticklabels=False),
        yaxis2=dict(
            title="Part Cumulée",
            overlaying="y",
            side="right",
            tickformat=".0%",
            range=[0, 1.1],
            showgrid=False,
        ),
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        barcornerradius=5,
    )

    fig5.add_annotation(
        x=0.99,
        y=-0.20,
        xref="paper",
        yref="paper",
        showarrow=False,
        align="right",
        text=f"Total Opportunité (Segment Alerte): <b>{total_opp:,.0f}</b> unités",
        font=dict(size=12, color="gray"),
    )

    fig5.show()


## 8) Export — tableau de bord HTML responsive

Export des **4 graphiques clés** (Segmentation, Performance, Mix produit, Plan d’action) dans un HTML responsive, pratique pour un portfolio (GitHub Pages / lien direct).


In [12]:
# -------------------------
# Responsive HTML template
# -------------------------
html_template = """
<!DOCTYPE html>
<html lang="fr">
<head>
<meta charset="utf-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<title>Retail Analytics Dashboard</title>
<link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;600;700&display=swap" rel="stylesheet">
<script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
<style>
    html, body {{
        margin: 0;
        padding: 0;
        width: 100%;
        overflow-x: hidden; /* prevent horizontal scrolling */
        background-color: #ffffff;
        font-family: 'Inter', sans-serif;
    }}

    .dashboard-container {{
        width: 100%;
        max-width: 100%;
        padding: 0 10px;
        box-sizing: border-box;
    }}

    h2 {{
        color: #1e293b;
        margin-top: 50px;
        margin-bottom: 20px;
        font-weight: 700;
        font-size: 20px;
        padding-left: 10px;
        border-left: 4px solid #3b82f6;
    }}

    .chart-wrapper {{
        width: 100%;
        height: 520px;
        margin-bottom: 40px;
        border: 1px solid transparent;
    }}

    hr {{ border: 0; border-top: 1px solid #f1f5f9; margin: 40px 0; }}

    .footer {{
        text-align: center;
        color: #94a3b8;
        font-size: 11px;
        margin-top: 60px;
        padding-bottom: 20px;
    }}
</style>
</head>
<body>

<div class="dashboard-container">

    <h2>1. Segmentation Stratégique</h2>
    <div id="chart1" class="chart-wrapper">
        {{chart1}}
    </div>

    <hr>

    <h2>2. Performance Structurelle</h2>
    <div id="chart2" class="chart-wrapper">
        {{chart2}}
    </div>

    <hr>

    <h2>3. Diagnostic Mix Produit</h2>
    <div id="chart4" class="chart-wrapper">
        {{chart4}}
    </div>

    <hr>

    <h2 style="border-left-color: #ef4444;">4. Plan d'Action (Priorités)</h2>
    <div id="chart5" class="chart-wrapper">
        {{chart5}}
    </div>

    <div class="footer">
        Retail Analytics Case • Rafael Midolli
    </div>

</div>

</body>
</html>
"""

# Make the figures fluid inside the HTML layout
for fig in [fig1, fig2, fig4, fig5]:
    fig.update_layout(
        width=None,
        height=None,
        autosize=True,
        margin=dict(l=10, r=10, t=40, b=40),
        paper_bgcolor="rgba(0,0,0,0)",
        plot_bgcolor="rgba(0,0,0,0)",
    )

print("Exporting responsive dashboard...")

html_fig1 = fig1.to_html(full_html=False, include_plotlyjs=False, config={"responsive": True, "displayModeBar": False})
html_fig2 = fig2.to_html(full_html=False, include_plotlyjs=False, config={"responsive": True, "displayModeBar": False})
html_fig4 = fig4.to_html(full_html=False, include_plotlyjs=False, config={"responsive": True, "displayModeBar": False})
html_fig5 = fig5.to_html(full_html=False, include_plotlyjs=False, config={"responsive": True, "displayModeBar": False})

final_html = (
    html_template.replace("{{chart1}}", html_fig1)
    .replace("{{chart2}}", html_fig2)
    .replace("{{chart4}}", html_fig4)
    .replace("{{chart5}}", html_fig5)
)

# Write output
REPORTS_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DASHBOARD.write_text(final_html, encoding="utf-8")

print(f"✅ Dashboard saved: {OUTPUT_DASHBOARD}")


Exporting responsive dashboard...
✅ Dashboard saved: ..\reports\dashboard_retail.html
